In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays 
using FileIO
using JLD2
using Plots
using LaTeXStrings
using DataFrames, Colors
using StatsPlots   

include("Params_PLD.jl")
include("Data_Generation_PLD.jl")
include("Estimation_PLD.jl")
include("Estimation_PLD_Fast.jl")
include("Models_PLD.jl")
include("Evaluation_PLD.jl")
include("Implement_All_Methods_PLD.jl")
include("Figures_PLD.jl")

hist_profit_distribution (generic function with 1 method)

In [2]:
Params = get_default_params_PLD()
# Params = get_Wang_Qi_Shen_params_PLD()
N = Params["N"] # number of products
N_x = Params["N_x"] # dimension of product features
c_l = Params["c_l"] 
d_r = Params["d_r"]
rev_gap = Params["rev_gap"]
N_u = Params["N_u"] # dimension of customer features
S_test = Params["S_test"] # test data size
N_Max = Params["N_Max"] # maximum assortment size
N_nonzero = Params["N_nonzero"] # number of nonzero entries in A
Time_Limit = Params["Time_Limit"] # time limit for optimization
dual_norm = Params["dual_norm"] # dual norm for robust optimization
norm_bounds = Params["norm_bounds"]
gamma_list = Params["gamma_list"] # list of gamma values for robust optimization
psi_lb = Params["psi_lb"] # lower bound for psi
psi_ub = Params["psi_ub"] # upper bound for psi
phi_lb = Params["phi_lb"]   # lower bound for phi
phi_ub = Params["phi_ub"]  # upper bound for phi
num_c = Params["num_c"] # number of customer segments
instances = Params["instances"] # number of instances
seed = Params["seed"] # random seed
coef_para_Input = Params["coef_this"] # coefficient for data generation
# coef_Wang_Qi_Shen = Params["coef_Wang_Qi_Shen"] # coefficient for Wang, Qi, Shen data generation

(alp0_lb = 1.0, alp0_ub = 2.0, alp_lb = -1.0, alp_ub = 0.0, beta_lb = -2.0, beta_ub = 2.0, A_lb = -2.0, A_ub = 2.0, r0_lb = 0.0, r0_ub = 1.0, r_lb = 0.0, r_ub = 0.1)

In [ ]:
# N_x = 4
# c_l = ones(N_x)

In [3]:
S_train_list = Params["S_train_all"] # training data size
is_ridge = Params["is_ridge"] # whether to use ridge regression

true

In [4]:
Random.seed!(seed)
is_Wang_Qi_Shen = true;
is_same_util_para = true;
if is_Wang_Qi_Shen
    project_dir = "BB_Wang_Qi_Shen_N=$(N)_N_x=$(N_x)_N_u=$(N_u)_N_nonzero=$(N_nonzero)_dr=$(d_r[1])_seed=$(seed)"
else
    project_dir = "BB_N=$(N)_N_x=$(N_x)_N_u=$(N_u)_N_nonzero=$(N_nonzero)_dr=$(d_r[1])_seed=$(seed)"
end
if is_same_util_para
    println("Generate data with the same utility parameters for all instances.")
    theta_true_Fixed, r_params_Fixed = Generate_Wang_Qi_Max_True_Paras(N_x,N_u,N_nonzero,coef_para_Input);
    project_dir = string(project_dir, "_Same_Util_Para/")
else
    println("Generate data with different utility parameters for all instances.")
    project_dir = string(project_dir, "_Diff_Util_Para/")
end
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/Product_Line_Design/")

data_dir = string(data_dir,project_dir)
if !isdir(data_dir)
    mkpath(data_dir)
end
println("Data directory: ", data_dir)
save(string(data_dir, "Params.jld2"), Params);

Generate data with the same utility parameters for all instances.
Data directory: /Users/zhangxun/Codes/Data/Product_Line_Design/BB_Wang_Qi_Shen_N=3_N_x=8_N_u=10_N_nonzero=20_dr=2.0_seed=12_Same_Util_Para/


In [5]:
function compute_w(params,z_input)
    alpha0 = params.alpha0
    alpha = params.alpha
    beta = params.beta
    A = params.A
    nu0 = alpha0 + beta' * z_input;
    nu = alpha .+ A * z_input;
    return nu0,nu
end

compute_w (generic function with 1 method)

In [6]:
function output_results(S_train,lambda,data_dir,instances,fig_display)
    Input_Data = load(string(data_dir, "Input_Data_S=$(S_train)_lambda=$(lambda).jld2"));
    RST_True_All = load(string(data_dir, "RST_True_S=$(S_train)_lambda=$(lambda).jld2"));
    RST_ETO_All = load(string(data_dir, "RST_ETO_S=$(S_train)_lambda=$(lambda).jld2"));
    RST_RO_All = load(string(data_dir, "RST_RO_S=$(S_train)_lambda=$(lambda).jld2"));

    gamma_list = sort([parse(Float64, split(k, "=")[end]) for k in keys(RST_RO_All["ins=1"])])
    gamma_list = gamma_list
    # println("Gamma list: ", gamma_list)

    obj_True, obj_ETO, obj_RO = obtain_obj(RST_True_All, RST_ETO_All, RST_RO_All, instances, gamma_list);
    println("S=$(S_train),lambda=$(lambda),obj True:",round.(mean(obj_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),obj ETO:",round.(mean(obj_ETO),digits=4))
    println("S=$(S_train),lambda=$(lambda),obj RO:",round.(mean(obj_RO,dims=1),digits=4))
    println()
    profit_True, profit_ETO, profit_RO = obtain_profits(RST_True_All, RST_ETO_All, RST_RO_All, instances, gamma_list);
    println("S=$(S_train),lambda=$(lambda),profit True:",round.(mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit ETO/True:",round.(mean(profit_ETO)/mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit RO/True:",round.(mean(profit_RO,dims=1)./mean(profit_True),digits=4))
    println("S=$(S_train),lambda=$(lambda),profit RO/ETO:",round.(mean(profit_RO,dims=1)./mean(profit_ETO),digits=4))
    # fig_name = string(data_dir, "RPLD_vs_ETOPLD_S=$(S_train)_lambda=$lambda.pdf")
    # include_std = false
    # line_plot_RPLD_vs_ETOPLD(profit_ETO,profit_RO,gamma_list,include_std,fig_name,fig_display)
end

output_results (generic function with 1 method)

In [7]:
Random.seed!(seed)
S_train = 200
lambda = 0.01
if is_same_util_para
    Input_Data_this = Generate_Data_this_Same_Para(N_Max,N_x,N_u,S_train,S_test,theta_true_Fixed, r_params_Fixed);
else
    Input_Data_this = Generate_Data_this(N_x,N_u,N_nonzero,S_train,S_test,m,coef_para_Input)
end
theta_true,r_params,X_train,Y_train,Z_train,Asorrtment_train,X_test,Y_test,Z_test = Get_Input_Data(Input_Data_this);
nu0_true,nu_true = compute_w(theta_true,Z_test[1,:])  
nu_all_true = [nu0_true;nu_true]


theta_hat = Estimation_This(N_Max,N_x,N_u,Y_train,X_train,Z_train, Asorrtment_train,is_ridge, lambda)
nu0_hat,nu_hat = compute_w(theta_hat,Z_test[1,:])  
nu_all_hat = [nu0_hat;nu_hat]

# ******** True Model *************
theta_Input = theta_true
RST_True,status_True = solve_ETO_This(S_test,N,N_x,theta_Input,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test)
# println("Oracle: status = ",status_True,",obj=",RST_True["obj"][1])
if status_True != "OPTIMAL"
    println("Warning: The true model did not reach optimality")
end

# ******** ETO Model *************
RST_ETO,status_ETO = solve_ETO_This(S_test,N,N_x,theta_hat,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test)
# println("ETO: status = ",status_ETO,",obj=",RST_ETO["obj"][1])
if status_ETO != "OPTIMAL"
    println("Warning: The ETO model did not reach optimality")
end

# # ******** RO Model *************
RST_RO_this = Dict()
gamma = gamma_list[2]
RST_RO,status_RO = solve_RO_this(S_test,N,N_x,theta_hat,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test,gamma,psi_lb,psi_ub,phi_lb,phi_ub)
# println("gamma = $gamma, RO: status = ",status_RO,",obj=",RST_RO["obj"][1])

(Dict{Any, Any}("X" => [0.0 1.0 0.0;;; 1.0 0.0 0.0;;; 0.0 0.0 0.0;;; 1.0 1.0 1.0;;; 1.0 1.0 1.0;;; 0.0 0.0 0.0;;; 0.0 0.0 0.0;;; 0.0 0.0 0.0], "obj" => [0.6560298867694817], "time" => [28.151933193206787], "status" => ["OPTIMAL"], "profit" => [0.6550999189093968]), "OPTIMAL")

## Branch and Bound

In [ ]:
# 定义创建优化器的函数
function create_optimizer(Time_Limit)
    optimizer = Mosek.Optimizer
    set_attribute(optimizer, "QUIET", true)
    set_optimizer_attribute(optimizer, "MSK_DPAR_OPTIMIZER_MAX_TIME", Time_Limit) 
    return optimizer
end

# 定义初始化模型的函数
function RO_PLD_Model(N, N_x, nu0, nu, r0, r, c_l, d_r, rev_gap, psi_lb, psi_ub, phi_lb, phi_ub, gamma, dual_norm, num_c, Time_Limit)
    # model = Model(create_optimizer(Time_Limit))
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    set_optimizer_attribute(model, "MSK_DPAR_OPTIMIZER_MAX_TIME", Time_Limit) 
    
    @variable(model, delta)          
    @variable(model, eta0 >= 0)              
    @variable(model, lbd0)            
    @variable(model, xi0[1:N_x])   
    @variable(model, eta[1:N] >= 0) 
    @variable(model, lbd[1:N])            
    @variable(model, xi[1:N, 1:N_x]) 
    
    # 定义指数锥变量
    @variable(model, psi_1[1:N])                   
    @variable(model, psi_2[1:N])                   
    @variable(model, psi_3[1:N])        
    @variable(model, phi_1[1:N])                   
    @variable(model, phi_2[1:N])                   
    @variable(model, phi_3[1:N])         

    @variable(model, X[1:N, 1:N_x], Bin)        # 二进制变量 x_{jk}
    @variable(model, Y[1:N,1:N_x])    
    @variable(model, Z[1:N,1:N_x])    

    # 定义约束
    @constraint(model, lbd0 + sum(psi_3) .== 0)
    @constraint(model, xi0 .+ sum(Y, dims=1)[1, :] .== 0)
    
    for ind1 in 1:N
        for ind2 in 1:N_x
            @constraint(model, Y[ind1, ind2] >= psi_lb[ind1] * X[ind1, ind2])
            @constraint(model, Y[ind1, ind2] <= psi_ub[ind1] * X[ind1, ind2])
            @constraint(model, Y[ind1, ind2] >= psi_3[ind1] - psi_ub[ind1] * (1 - X[ind1, ind2]))
            @constraint(model, Y[ind1, ind2] <= psi_3[ind1] - psi_lb[ind1] * (1 - X[ind1, ind2]))
        end
    end
    
    # 添加其他约束...
    # 省略其他约束的定义

    return model
end

In [ ]:
# 求解松弛问题的函数
function solve_relaxed_problem(model)
    optimize!(model)
    if termination_status(model) == MOI.OPTIMAL
        return objective_value(model)
    else
        return nothing  # 如果没有找到最优解，返回nothing
    end
end

In [ ]:
# 选择分支变量
function choose_branching_variable(model, N)
    best_variable = nothing
    min_gap = Inf
    for n in 1:N
        for k in 1:size(model.X, 2)
            value = value(model.X[n, k])
            gap = abs(value - 0.5)  # 选择最接近0.5的二进制变量
            if gap < min_gap
                min_gap = gap
                best_variable = (n, k)
            end
        end
    end
    return best_variable
end

In [ ]:
# 分支操作
function branch(model, branching_variable, value)
    subproblem_left = copy(model)
    subproblem_right = copy(model)
    
    n, k = branching_variable
    if value == 0
        @constraint(subproblem_left, model.X[n, k] == 0)
        @constraint(subproblem_right, model.X[n, k] == 1)
    else
        @constraint(subproblem_left, model.X[n, k] == 1)
        @constraint(subproblem_right, model.X[n, k] == 0)
    end
    
    return subproblem_left, subproblem_right
end

In [ ]:
nu0 = nu0_hat
nu = nu_hat
r0, r = r_params
max_iterations = 2

In [ ]:
best_solution = nothing
best_upper_bound = Inf
open_nodes = [RO_PLD_Model(N, N_x, nu0, nu, r0, r, c_l, d_r, rev_gap, psi_lb, psi_ub, phi_lb, phi_ub, gamma, dual_norm, num_c, Time_Limit)]

iterations = 0
while !isempty(open_nodes) && iterations < max_iterations
    node = pop!(open_nodes)  # 获取下一个节点
    
    # 求解松弛问题
    relaxed_solution = solve_relaxed_problem(node)
    if relaxed_solution == nothing
        continue  # 如果无解，跳过
    
    # 更新最优解
    if relaxed_solution < best_upper_bound
        best_upper_bound = relaxed_solution
        best_solution = get_solution(node)
    end
    
    # 分支
    branching_variable = choose_branching_variable(node, N)
    subproblem_left, subproblem_right = branch(node, branching_variable, 0)
    push!(open_nodes, subproblem_left, subproblem_right)
    
    iterations += 1
end

In [ ]:
# 分支定界算法
function branch_and_bound(N, max_iterations, Time_Limit)
    best_solution = nothing
    best_upper_bound = Inf
    open_nodes = [RO_PLD_Model(N, N_x, nu0, nu, r0, r, c_l, d_r, rev_gap, psi_lb, psi_ub, phi_lb, phi_ub, gamma, dual_norm, num_c, Time_Limit)]
    
    iterations = 0
    while !isempty(open_nodes) && iterations < max_iterations
        node = pop!(open_nodes)  # 获取下一个节点
        
        # 求解松弛问题
        relaxed_solution = solve_relaxed_problem(node)
        if relaxed_solution == nothing
            continue  # 如果无解，跳过
        
        # 更新最优解
        if relaxed_solution < best_upper_bound
            best_upper_bound = relaxed_solution
            best_solution = get_solution(node)
        end
        
        # 分支
        branching_variable = choose_branching_variable(node, N)
        subproblem_left, subproblem_right = branch(node, branching_variable, 0)
        push!(open_nodes, subproblem_left, subproblem_right)
        
        iterations += 1
    end
    
    return best_solution
end

## Branch and Bound, Genemi Version

In [8]:
using JuMP
using MosekTools
using LinearAlgebra
using DataStructures # 用于 Stack
using Printf

## Generate initial solution

In [9]:
function random_unique_binary_matrix(n_rows::Int, n_cols::Int; ones_per_row::Union{Nothing,AbstractVector{<:Integer}}=nothing)
    n_rows >= 1 || error("n_rows must be positive")
    n_cols >= 2 || error("n_cols must be at least 2 to place two ones")
    counts = ones_per_row === nothing ? nothing : Int.(ones_per_row)

    if counts === nothing
        max_unique_rows = 2^n_cols - (n_cols + 1)
        n_rows <= max_unique_rows || error("Cannot build $(n_rows) unique rows with at least two ones; maximum is $(max_unique_rows)")
    else
        length(counts) == n_rows || error("ones_per_row length must equal n_rows")
        needed_per_count = Dict{Int,Int}()
        for c in counts
            2 <= c <= n_cols || error("Each ones_per_row entry must lie in [2, n_cols]")
            current = get!(needed_per_count, c, 0) + 1
            needed_per_count[c] = current
            current <= binomial(n_cols, c) || error("Need $(current) distinct rows with $c ones but only $(binomial(n_cols, c)) combinations exist")
        end
    end

    result = zeros(Int, n_rows, n_cols)
    seen = Set{String}()
    for row_idx in 1:n_rows
        while true
            row = zeros(Int, n_cols)
            ones_count = counts === nothing ? rand(2:n_cols) : counts[row_idx]
            row[randperm(n_cols)[1:ones_count]] .= 1
            key = join(row, ",")
            if !(key in seen)
                push!(seen, key)
                result[row_idx, :] = row
                break
            end
        end
    end
    return result
end

function random_unique_binary_matrix(N::Int)
    random_unique_binary_matrix(N, N)
end

function random_unique_binary_matrices(n_rows::Int, n_cols::Int, count::Int=100; ones_per_row::Union{Nothing,AbstractVector{<:Integer}}=nothing)
    count > 0 || error("count must be positive")
    matrices = Vector{Matrix{Int}}()
    seen = Set{String}()
    while length(matrices) < count
        mat = random_unique_binary_matrix(n_rows, n_cols; ones_per_row=ones_per_row)
        key = join(vec(mat), ",")
        if !(key in seen)
            push!(seen, key)
            push!(matrices, mat)
        end
    end
    return matrices
end

function random_unique_binary_matrices(N::Int, count::Int=100; ones_per_row::Union{Nothing,AbstractVector{<:Integer}}=nothing)
    random_unique_binary_matrices(N, N, count; ones_per_row=ones_per_row)
end

random_unique_binary_matrices (generic function with 3 methods)

In [10]:
function RO_PLD_Given_Design(N,N_x,nu0,nu,r0,r,c_l,d_r,rev_gap,psi_lb,psi_ub,phi_lb,phi_ub,gamma,dual_norm,num_c,Time_Limit,X_given)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    set_optimizer_attribute(model, "MSK_DPAR_OPTIMIZER_MAX_TIME", Time_Limit) 
    # 定义变量
    @variable(model, delta)          
    @variable(model, eta0 >= 0)              
    @variable(model, lbd0)            
    @variable(model, xi0[1:N_x])   
    @variable(model, eta[1:N] >= 0) 
    @variable(model, lbd[1:N])            
    @variable(model, xi[1:N,1:N_x]) 

    # exponential variables
    @variable(model, psi_1[1:N])                   
    @variable(model, psi_2[1:N])                   
    @variable(model, psi_3[1:N])        
    @variable(model, phi_1[1:N])                   
    @variable(model, phi_2[1:N])                   
    @variable(model, phi_3[1:N])         

    @variable(model, X[1:N, 1:N_x],Bin)        # 二进制变量 x_{jk}
    @variable(model, Y[1:N,1:N_x])    
    @variable(model, Z[1:N,1:N_x])    

    @constraint(model, lbd0 + sum(psi_3) .== 0)

    @constraint(model, xi0 .+ sum(Y,dims=1)[1,:] .== 0)
    for ind1 in 1:N
        for ind2 in 1:N_x 
            @constraint(model, Y[ind1,ind2] >= psi_lb[ind1] * X[ind1,ind2])
            @constraint(model, Y[ind1,ind2] <= psi_ub[ind1] * X[ind1,ind2])
            @constraint(model, Y[ind1,ind2] >= psi_3[ind1] - psi_ub[ind1] * (1 - X[ind1,ind2]))
            @constraint(model, Y[ind1,ind2] <= psi_3[ind1] - psi_lb[ind1] * (1 - X[ind1,ind2]))
        end
    end

    for n in 1:N
        @constraint(model, lbd[n] - phi_3[n] == 0)
    end
    for n in 1:N
        @constraint(model, xi[n,:] .- Z[n,:] .== 0)
    end
    for ind1 in 1:N
        for ind2 in 1:N_x 
            @constraint(model, Z[ind1,ind2] >= phi_lb[ind1] * X[ind1,ind2])
            @constraint(model, Z[ind1,ind2] <= phi_ub[ind1] * X[ind1,ind2])
            @constraint(model, Z[ind1,ind2] >= phi_3[ind1] - phi_ub[ind1] * (1 - X[ind1,ind2]))
            @constraint(model, Z[ind1,ind2] <= phi_3[ind1] - phi_lb[ind1] * (1 - X[ind1,ind2]))
        end
    end

    @constraint(model, delta + sum(psi_2) + sum(phi_1) + eta0 * gamma - lbd0 * nu0 - nu' * xi0 == 0)

    for n in 1:N
        @constraint(model, delta + psi_1[n] + phi_2[n] + eta[n] * gamma - lbd[n] * nu0 - nu' * xi[n,:] - r0 - r' * X[n,:] == 0)
    end
    
    for n in 1:N
        @constraint(model, [psi_3[n], psi_2[n], psi_1[n]] in MOI.DualExponentialCone())
    end

    for n in 1:N
        @constraint(model, [phi_3[n],phi_2[n],phi_1[n]] in MOI.DualExponentialCone())
    end

    if dual_norm == 2
        @constraint(model, [eta0;lbd0;xi0] in SecondOrderCone())
        for n in 1:N
            @constraint(model, [eta[n];lbd[n];xi[n,:]] in SecondOrderCone())
        end
    end
    if dual_norm == 1
        @constraint(model, [eta0;lbd0;xi0] in MOI.NormOneCone(N_x+2))
        for n in 1:N
            @constraint(model, [eta[n];lbd[n];xi[n,:]] in MOI.NormOneCone(N_x+2))
        end
    end

    @constraint(model, X .- X_given .== 0)
    

    @constraint(model, X * c_l .>= d_r)
    for ind1 in 1:N
        @constraint(model, c_l' * Y[ind1,:] <= d_r[ind1] * psi_3[ind1])
        @constraint(model, c_l' * Z[ind1,:] <= d_r[ind1] * phi_3[ind1])
    end

    for ind1 in 1:(N-1)
        @constraint(model, r' * X[ind1,:] >= r' * X[(ind1+1),:] + rev_gap)
    end

    @objective(model, Max, delta)
    
    optimize!(model)
    status = JuMP.termination_status(model)
    # println("status: ", status)
    # solution_summary(model)
    if status == MOI.OPTIMAL  || status == MOI.TIME_LIMIT
        sol_status = string(status)
        obj_val = objective_value(model)
        X_val = round.(value.(X))
        solve_time = JuMP.solve_time(model)
    else
        sol_status = "Others"
        obj_val = NaN
        X_val = ones(N,N) .* NaN
        solve_time = NaN
    end
    return obj_val, X_val,solve_time,sol_status
end

RO_PLD_Given_Design (generic function with 1 method)

In [11]:
function solve_RO_greedy(N,N_x,nu0,nu,r0,r,c_l,d_r,rev_gap,psi_lb,psi_ub,phi_lb,phi_ub,gamma,dual_norm,num_c,Time_Limit,Matrices)
    obj_opt = 0
    x_sol = nothing
    start_time = time()
    status = ""
    for mat in Matrices
        obj_list1, x_list1, time_list1,status_list1 = RO_PLD_Given_Design(N,N_x,nu0,nu,r0,r,c_l,d_r,rev_gap,psi_lb,psi_ub,phi_lb,phi_ub,gamma,dual_norm,num_c,Time_Limit,mat)
        if obj_list1 > obj_opt
            obj_opt = obj_list1
            x_sol = x_list1
            status = status_list1
        end
    end
    solve_time = time() - start_time
    return obj_opt, x_sol, solve_time,status
end

solve_RO_greedy (generic function with 1 method)

In [12]:
# ==========================================
# 1. 定义数据结构
# ==========================================

# 节点结构：存储被固定的变量索引
struct BnBNode
    # 使用 CartesianIndex 来存储矩阵 X 的二维索引 (i, j)
    fixed_0::Vector{CartesianIndex{2}} 
    fixed_1::Vector{CartesianIndex{2}}
end

# 存储全局最优解
mutable struct Incumbent
    obj_val::Float64          # 当前最好的整数解目标值 (Lower Bound for Max problem)
    X_val::Matrix{Float64}    # 对应的 X 值
    found::Bool               # 是否找到了至少一个整数解
end

# ==========================================
# 2. 松弛求解函数 (基于你的 RO_PLD 修改)
# ==========================================

function solve_relaxed_model(node::BnBNode, N, N_x, nu0, nu, r0, r, c_l, d_r, 
                             rev_gap, psi_lb, psi_ub, phi_lb, phi_ub, gamma, 
                             dual_norm, num_c, Time_Limit)
    
    # 创建模型
    model = Model(Mosek.Optimizer)
    set_silent(model) # 关闭 Mosek 输出，避免 B&B 刷屏
    # 注意：这里的 Time_Limit 是针对单节点求解的限制，可以设稍微大一点
    set_optimizer_attribute(model, "MSK_DPAR_OPTIMIZER_MAX_TIME", Time_Limit)

    # --- 定义变量 (注意 X 改为连续区间 [0, 1]) ---
    @variable(model, delta)
    @variable(model, eta0 >= 0)
    @variable(model, lbd0)
    @variable(model, xi0[1:N_x])
    @variable(model, eta[1:N] >= 0)
    @variable(model, lbd[1:N])
    @variable(model, xi[1:N,1:N_x])
    
    # Exponential variables
    @variable(model, psi_1[1:N])
    @variable(model, psi_2[1:N])
    @variable(model, psi_3[1:N])
    @variable(model, phi_1[1:N])
    @variable(model, phi_2[1:N])
    @variable(model, phi_3[1:N])
    
    # *** 关键修改：X 松弛为 [0, 1] 之间的连续变量 ***
    @variable(model, 0 <= X[1:N, 1:N_x] <= 1) 
    
    @variable(model, Y[1:N,1:N_x])
    @variable(model, Z[1:N,1:N_x])

    # --- 添加约束 (逻辑与原模型一致) ---
    @constraint(model, lbd0 + sum(psi_3) .== 0)
    @constraint(model, xi0 .+ sum(Y,dims=1)[1,:] .== 0)

    for ind1 in 1:N
        for ind2 in 1:N_x
            # Big-M 约束在松弛下依然有效 (Convex Hull)
            @constraint(model, Y[ind1,ind2] >= psi_lb[ind1] * X[ind1,ind2])
            @constraint(model, Y[ind1,ind2] <= psi_ub[ind1] * X[ind1,ind2])
            @constraint(model, Y[ind1,ind2] >= psi_3[ind1] - psi_ub[ind1] * (1 - X[ind1,ind2]))
            @constraint(model, Y[ind1,ind2] <= psi_3[ind1] - psi_lb[ind1] * (1 - X[ind1,ind2]))
        end
    end

    for n in 1:N
        @constraint(model, lbd[n] - phi_3[n] == 0)
    end

    for n in 1:N
        @constraint(model, xi[n,:] .- Z[n,:] .== 0)
    end

    for ind1 in 1:N
        for ind2 in 1:N_x
            @constraint(model, Z[ind1,ind2] >= phi_lb[ind1] * X[ind1,ind2])
            @constraint(model, Z[ind1,ind2] <= phi_ub[ind1] * X[ind1,ind2])
            @constraint(model, Z[ind1,ind2] >= phi_3[ind1] - phi_ub[ind1] * (1 - X[ind1,ind2]))
            @constraint(model, Z[ind1,ind2] <= phi_3[ind1] - phi_lb[ind1] * (1 - X[ind1,ind2]))
        end
    end

    @constraint(model, delta + sum(psi_2) + sum(phi_1) + eta0 * gamma - lbd0 * nu0 - nu' * xi0 == 0)

    for n in 1:N
        @constraint(model, delta + psi_1[n] + phi_2[n] + eta[n] * gamma - lbd[n] * nu0 - nu' * xi[n,:] - r0 - r' * X[n,:] == 0)
    end

    for n in 1:N
        @constraint(model, [psi_3[n], psi_2[n], psi_1[n]] in MOI.DualExponentialCone())
    end

    for n in 1:N
        @constraint(model, [phi_3[n],phi_2[n],phi_1[n]] in MOI.DualExponentialCone())
    end

    if dual_norm == 2
        @constraint(model, [eta0;lbd0;xi0] in SecondOrderCone())
        for n in 1:N
            @constraint(model, [eta[n];lbd[n];xi[n,:]] in SecondOrderCone())
        end
    end

    if dual_norm == 1
        @constraint(model, [eta0;lbd0;xi0] in MOI.NormOneCone(N_x+2))
        for n in 1:N
            @constraint(model, [eta[n];lbd[n];xi[n,:]] in MOI.NormOneCone(N_x+2))
        end
    end

    @constraint(model, X * c_l .>= d_r)

    for ind1 in 1:N
        @constraint(model, c_l' * Y[ind1,:] <= d_r[ind1] * psi_3[ind1])
        @constraint(model, c_l' * Z[ind1,:] <= d_r[ind1] * phi_3[ind1])
    end

    for ind1 in 1:(N-1)
        @constraint(model, r' * X[ind1,:] >= r' * X[(ind1+1),:] + rev_gap)
    end

    @objective(model, Max, delta)

    # --- 应用 B&B 分支约束 (Fix Variables) ---
    for idx in node.fixed_0
        fix(X[idx], 0.0; force=true)
    end
    for idx in node.fixed_1
        fix(X[idx], 1.0; force=true)
    end

    # --- 求解 ---
    optimize!(model)
    status = termination_status(model)

    if status == MOI.OPTIMAL
        return (status=:Optimal, obj=objective_value(model), X=value.(X))
    elseif status == MOI.INFEASIBLE || status == MOI.INFEASIBLE_OR_UNBOUNDED
        return (status=:Infeasible, obj=-Inf, X=zeros(N, N_x))
    else
        # 处理超时或数值错误
        return (status=:Error, obj=-Inf, X=zeros(N, N_x))
    end
end

# ==========================================
# 3. 分支定界主算法
# ==========================================

function run_branch_and_bound(N, N_x, nu0, nu, r0, r, c_l, d_r, rev_gap, 
                              psi_lb, psi_ub, phi_lb, phi_ub, gamma, 
                              dual_norm, num_c, Time_Limit,
                              obj_given, x_given)
    
    # 计时开始
    t_start = time()

    # 初始化栈 (深度优先搜索 DFS)
    stack = Stack{BnBNode}()
    push!(stack, BnBNode(CartesianIndex{2}[], CartesianIndex{2}[]))

    # 初始化全局最优解 (Max 问题，初始为负无穷)
    # 如果你有启发式算法可以算出一个初始可行解，可以在这里填入，能加速剪枝
    # best_sol = Incumbent(-Inf, zeros(N, N_x), false)
    best_sol = Incumbent(obj_given, zeros(N, N_x), false)

    iter = 0
    node_count = 0
    int_tol = 1e-5 # 整数判断容差

    println("=================================================================================")
    println(" Starting Branch and Bound for MIECP (Max Problem)")
    println("=================================================================================")
    println(" Iter | Nodes | Best Integer Obj (LB) | Current Node Obj (UB) | Action")
    println("---------------------------------------------------------------------------------")

    while !isempty(stack)
        # 检查总时间限制
        if time() - t_start > Time_Limit
            println(">>> Global Time Limit Exceeded!")
            break
        end

        iter += 1
        node = pop!(stack) # 取出一个节点
        
        # 1. 求解松弛问题
        res = solve_relaxed_model(node, N, N_x, nu0, nu, r0, r, c_l, d_r, 
                                  rev_gap, psi_lb, psi_ub, phi_lb, phi_ub, gamma, 
                                  dual_norm, num_c, Time_Limit)

        action = ""
        
        # 2. 剪枝逻辑
        if res.status != :Optimal
            # 不可行或出错，剪枝
            action = "Prune (Infeasible)"
        elseif res.obj <= best_sol.obj_val + 1e-6
            # Bound Pruning (Max问题): 
            # 如果当前松弛解(UB) <= 目前最好的整数解(LB)，则不可能找到更优解，剪枝
            action = "Prune (Bound)"
        else
            # 松弛解比当前最好解要好 (Potential for improvement)
            
            # 检查整数可行性
            fractional_vars = CartesianIndex{2}[]
            for i in 1:N, j in 1:N_x
                val = res.X[i, j]
                if abs(val - 0.0) > int_tol && abs(val - 1.0) > int_tol
                    push!(fractional_vars, CartesianIndex(i, j))
                end
            end

            if isempty(fractional_vars)
                # 所有 X 都是整数 -> 找到更好的可行解 (New Incumbent)
                best_sol.obj_val = res.obj
                best_sol.X_val = res.X
                best_sol.found = true
                action = "*** New Best Integer Sol! ***"
            else
                # 存在分数的变量 -> 分支 (Branching)
                
                # 策略：Most Fractional (选择最接近 0.5 的变量)
                sort!(fractional_vars, by = idx -> abs(res.X[idx] - 0.5))
                branch_idx = fractional_vars[1] # 选差异最大的

                action = "Branch on X[$(branch_idx[1]), $(branch_idx[2])]"

                # 生成两个子节点并推入栈
                # 左子节点: X = 0
                push!(stack, BnBNode([node.fixed_0; branch_idx], node.fixed_1))
                # 右子节点: X = 1
                push!(stack, BnBNode(node.fixed_0, [node.fixed_1; branch_idx]))
                
                node_count += 2
            end
        end

        # 打印日志 (每 10 次或找到新解时打印)
        if iter % 10 == 1 || occursin("New Best", action)
            @Printf.printf("%5d | %5d | %20.6f | %20.6f | %s\n", 
                           iter, length(stack), best_sol.obj_val, res.obj, action)
        end
    end

    println("---------------------------------------------------------------------------------")
    println("Optimization Finished.")
    
    status_str = best_sol.found ? "Optimal (Integer)" : "Infeasible"
    total_time = time() - t_start

    if best_sol.found
        println("Best Objective: ", best_sol.obj_val)
        return best_sol.obj_val, round.(best_sol.X_val), total_time, status_str
    else
        println("No integer solution found.")
        return NaN, fill(NaN, N, N_x), total_time, status_str
    end
end

run_branch_and_bound (generic function with 1 method)

In [13]:
nu0 = nu0_hat
nu = nu_hat
r0, r = r_params
max_iterations = 2

2

In [18]:
Num_Mat = 10000;
Matrices = random_unique_binary_matrices(N, N_x, Num_Mat);
obj_initial, x_initial, time_initial,status_initial = solve_RO_greedy(N,N_x,nu0,nu,r0,r,c_l,d_r,rev_gap,psi_lb,psi_ub,phi_lb,phi_ub,gamma,dual_norm,num_c,Time_Limit,Matrices);

In [19]:
obj_initial

0.6451498723795114

In [20]:
run_branch_and_bound(N, N_x, nu0, nu, r0, r, c_l, d_r, rev_gap, 
                              psi_lb, psi_ub, phi_lb, phi_ub, gamma, 
                              dual_norm, num_c, Time_Limit,
                              obj_initial, x_initial)

(NaN, [NaN NaN … NaN NaN; NaN NaN … NaN NaN; NaN NaN … NaN NaN], 300.003130197525, "Infeasible")

In [17]:
RST_RO

Dict{Any, Any} with 5 entries:
  "X"      => [0.0 1.0 0.0;;; 1.0 0.0 0.0;;; 0.0 0.0 0.0;;; 1.0 1.0 1.0;;; 1.0 …
  "obj"    => [0.65603]
  "time"   => [28.1519]
  "status" => ["OPTIMAL"]
  "profit" => [0.6551]